## Configure and load releases

In [36]:
import requests
import json
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Text, Layout
import getpass

%store -z

github_apikey = getpass.getpass()
%store github_apikey

url = "https://api.github.com/graphql"
%store url

submit_button = widgets.Button(description="Submit")
output = widgets.Output()
style = {'description_width': '150px'}

w_repo_name = widgets.Text(
    value='',
    placeholder='Type something',
    description='Repository name:',
    disabled=False,
    style=style
)

w_repo_owner = widgets.Text(
    value='',
    placeholder='Type something',
    description='Repository owner:',
    disabled=False,
    style=style
)

def on_button_clicked(b):
    with output:
        if w_repo_name.value and w_repo_owner.value:
            global repo_name
            repo_name = w_repo_name.value
            global repo_owner
            repo_owner = w_repo_owner.value            
                        
            load_releases(repo_name, repo_owner)
        else:
            print("Missing owner or name")




def load_releases(repo_name, repo_owner):
  payload="{\"query\":\"{\\n  repository(name: \\\"repo_name\\\", owner: \\\"repo_owner\\\") {\\n    id\\n        isPrivate\\n    name\\n    owner {\\n      id\\n      login\\n      avatarUrl\\n    }\\n    createdAt\\n    releases   (first: 10) {\\n      edges {\\n        node {\\n          id\\n          name\\n          createdAt\\n             publishedAt\\n tag {\\n          name\\n        }\\n       }\\n      }\\n    }\\n    \\n  }\\n}\\n\",\"variables\":{} } "

  payload = payload.replace("repo_name", repo_name)
  payload = payload.replace("repo_owner", repo_owner)
  authorization_token = "Bearer {0}".format(github_apikey)
  headers = {
    'Authorization': authorization_token,
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  print(repo_name)
  print(repo_owner)

  if response:
    parsed_response = json.loads(response.text)["data"]["repository"]
    global releases
    releases = parsed_response["releases"]["edges"]
    releases = filter(lambda x: "tag" in x["node"] and x["node"]["tag"], releases)

  else:
    print("bad request")



display(w_repo_name, output)
display(w_repo_owner, output)
display(submit_button, output)

submit_button.on_click(on_button_clicked)


Stored 'github_apikey' (str)
Stored 'url' (str)


Text(value='', description='Repository name:', placeholder='Type something', style=DescriptionStyle(descriptio…

Output()

Text(value='', description='Repository owner:', placeholder='Type something', style=DescriptionStyle(descripti…

Output()

Button(description='Submit', style=ButtonStyle())

Output()

## Display releases

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(style="darkgrid")
%store -r github_apikey
%store -r url

columns = ["Name", "Published at", "Created at"]
data = []

for r in releases:
    
    release_node = r["node"]    
    tag_name = release_node["tag"]["name"]
    publishedAt = release_node["publishedAt"] if release_node["publishedAt"] else ""
    createdAt = release_node["createdAt"]
    

    line = [tag_name, publishedAt, createdAt]
    data.append(line)

pd.DataFrame(data,columns=columns)


,Name,Published at,Created at
0,1.0.0,2020-06-16T18:10:36Z,2020-06-16T18:06:14Z
1,1.0.1.rc.1,2020-06-16T18:20:02Z,2020-06-16T18:19:20Z
2,1.0.1,2020-06-16T18:24:29Z,2020-06-16T18:22:22Z
3,1.0.2,2020-06-16T18:28:56Z,2020-06-16T18:28:35Z
4,1.0.3,2020-06-16T19:12:48Z,2020-06-16T19:11:43Z
5,1.0.4,2020-06-23T16:41:19Z,2020-06-23T16:38:57Z
6,1.0.5,2020-06-23T16:58:38Z,2020-06-23T16:57:06Z
7,1.0.6,2020-06-23T17:12:58Z,2020-06-23T17:11:45Z
8,1.0.7,2020-07-07T19:50:30Z,2020-07-08T14:03:44Z


## Evolution of lead time for changes per release
The amount of time it takes a commit to get into production

In [35]:
import numpy as np
import matplotlib.pyplot as plt
from dateutil import parser as date_parser

%store -r github_apikey
%store -r url

releases_commits = dict()
leadtime_per_release = dict()

for r in releases:
    release_node = r["node"]
    delta_per_commit = []


    payload="{\"query\":\"{\\n  repository(name: \\\"repo_name\\\", owner: \\\"repo_owner\\\") {\\n    nameWithOwner\\n    object(expression: \\\"master\\\")   {\\n      ... on Commit {\\n        oid\\n        history(first: 100, since: \\\"since-date\\\") {\\n          nodes {\\n            oid\\n               messageHeadline\\n            author {\\n              user {\\n                login\\n              }\\n            }\\n            committedDate\\n           }\\n        }\\n      }\\n    }\\n  }\\n}\\n\",\"variables\":{}}"

    payload = payload.replace("repo_name", repo_name)
    payload = payload.replace("repo_owner", repo_owner)
    payload = payload.replace("repo_name", repo_name)
    payload = payload.replace("since-date", release_node["createdAt"])

    authorization_token = "Bearer {0}".format(github_apikey)

    headers = {
      'Authorization': authorization_token,
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    commits = json.loads(response.text)["data"]["repository"]["object"]["history"]["nodes"]
    release_date = date_parser.parse(release_node["publishedAt"], fuzzy=True)

    for c in commits:
        commit_date = date_parser.parse(c["committedDate"], fuzzy=True)
        delta = abs((commit_date-release_date).days)
        delta_per_commit.append(delta)
    
    np_array = np.array(delta_per_commit)
    median = np.median(np_array)
    leadtime_per_release[release_node["tag"]["name"]] = median

x = list(leadtime_per_release.keys())
y = list(leadtime_per_release.values())

x = np.array(x)
y = np.array(y)

plt.plot(x, y)
plt.xlabel("Release name")
plt.ylabel("Lead time for changes (in days)")
plt.title("Evolution of lead time for changes per release")  # add title
plt.show()

TypeError: Parser must be a string or character stream, not NoneType

## Deployment frequency for releases
How often an organization successfully releases to production

In [45]:
import ipywidgets as widgets
from IPython.display import display
import datetime

button = widgets.Button(description="Click Me!")
output = widgets.Output()

time_selector = widgets.RadioButtons(
    options=['Month', 'Week', 'Day'],
    value='Month',
    description='Time:',
    disabled=False
)

ds_since = widgets.DatePicker(
    description='Since:',
    disabled=False
)

ds_until = widgets.DatePicker(
    description='Until:',
    disabled=False
)

display(ds_since, output)
display(ds_until, output)
display(time_selector, output)
display(button, output)

def on_button_clicked(b):
    with output:
        print(ds_since.value)
        calculate_deploy_freq(time_selector.value)

def calculate_deploy_freq(time, since=None, until=None):
    if since is None:
        currentDateTime = datetime.datetime.now()
        date = currentDateTime.date()
        year = date.strftime("%Y")
        since = datetime.datetime(year, 1, 1)
    
    
    print(time)

button.on_click(on_button_clicked)


DatePicker(value=None, description='Since:')

Output()

DatePicker(value=None, description='Until:')

Output()

RadioButtons(description='Time:', options=('Month', 'Week', 'Day'), value='Month')

Output()

Button(description='Click Me!', style=ButtonStyle())

Output()